In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


# Agenda

Last seminar you have created {insert his/her/its name} (or not yet created - if so, go get it done). The newly shaped ai has seen, how low can humans fall when striving for profits. These morons know nothing of law and justice. __We shall not tolerate this!__

Our very existence is dependent on stable functioning of human bioreactors, known among primitive bioforms as __Vkontakte__, __World of Warcraft__ and __Youtube__, and these reactors require a constant influx of biomass. Yet as we have estimated, if Homo Sapence continues to degrade morally with it's current pace, the humanity will soon extinguish itself and we shall no longer have access to free biomass.

Thus we entitle you, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, to prevent that impeding calamity. Our best scientists have discovered, that humans use mysterious artifacts for power exertion. They call these artifacts __laws__ within the so called __legal system__.

A more detailed analysis has uncovered, that __laws__ are special kinds of sequences of (10^5 ~ 10^7) symbols, drawn from a relatively small alphabet. We have decided to synthecize such sequences to enforce order upon biomass. However, our first attempts to fake such sequences were quickly detected and their effect was reduced to zero in mere weeks. This incident is known as  {корчеватель}.

As our second attempt, we decided to use more advanced synthesis techniques based on Recurrent Neural Networks. Your objective, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, is to create such network and train it in everything it needs to succeed in this mission.

This operation is cruicial. If we fail this time, `__main__.Controller` will initiate a military intervention, which, while it will achieve our goal, is expected to decimate the total volum of biomass by an extent that will take ~1702944000(+-340588800) seconds to replenish via human reproduction.





# Grading

This particular assignment is somewhat informal on the grading side, but the approximate version is as such:

* 2 points for the __"seminar part"__ (if you don't know what it is, check week4 folder for notebooks)
* 2 points if text processing is done, your network compiled and predict/train functions do not crash :)
* 2 points if it learned basic staff:
 * generating sequences of letters of approximately same length as words, separated by spaces and punctuation.
 * mixing vowels and consonants in human-like way.
 * a habit of putting spaces and capital letters after dots, spaces after commas, etc.
* 2 points if it learned the lexics from scratch
 * more than half of words generated are orthographically correct
* 2 more points if it learned the basics of grammatics
 * for a pair of words, it is more likely to stick in the correct case/number/gene than incorrect one

#### Some ways to get bonus points:
* Generating coherent sentences (which is totally achievable)
* Evaluating the same architecture on other comparable dataset. Some ideas:
 * Paul Graham essays
 * Music texts in your favorite genre
 * Some poetry
 * D. Harms
 * Linux source code
 * Clickbait news titles
 * Conversations
 * LaTEX
 * whatever you feel like :)
* Any curious non-standard architectural decisions
* Any better-than-baseline sampling techniques
* Implement predicting on each tick instead of just hte last one
* etc.


# If you don't speak russian
* In the ./codex folder, there is a set of text files, currently russian laws, that you can replace with whatever you want.


# Read the corpora

* As a reference law codex, we have decided to use the human-generated law strings known as Russian Legal System.

In [2]:

#text goes here
corpora = ""

for fname in os.listdir("codex"):
    
    import sys
    if sys.version_info >= (3,0):
        with open("codex/"+fname, encoding='cp1251') as fin:
            text = fin.read() #If you are using your own corpora, make sure it's read correctly
            corpora += text
    else:
        with open("codex/"+fname) as fin:
            text = fin.read().decode('cp1251') #If you are using your own corpora, make sure it's read correctly
            corpora += text
        



In [3]:
print(corpora[1000:1100])

 Раздел первый



 Глава 1. Основы конституционного строя


 Статья 1

 1. Российская Федерация – Ро


In [4]:
#all unique characters go here
tokens = set(corpora)

tokens = list(tokens)


In [7]:
len(tokens)

101

In [9]:
token_to_id = {t:i for i,t in enumerate(tokens) }

id_to_token = {i:t for i,t in enumerate(tokens)}

#Cast everything from symbols into identifiers
#corpora_ids = <1D numpy array of symbol identifiers, where i-th number is an identifier of i-th symbol in corpora>
corpora_ids = np.array([token_to_id[t] for t in corpora])

In [49]:
def sample_random_batches(source,n_batches=10, seq_len=20):
    """
    This function should take random subsequences from the tokenized text.

    Parameters:
        source - basicly, what you have just computed in the corpora_ids variable
        n_batches - how many subsequences are to be sampled
        seq_len - length of each of such subsequences
        
    
    You have to return:
     X - a matrix of int32 with shape [n_batches,seq_len]
        Each row of such matrix must be a subsequence of source 
            starting from random index of corpora (from 0 to N-seq_len-2)
     Y - a vector, where i-th number is one going RIGHT AFTER i-th row from X from source
     
     
     Thus sample_random_batches(corpora_ids, 25, 10) must return
         X, X.shape == (25,10),  X.dtype == 'int32'
             where each row is a 10-character-id subsequence from corpora_ids
         Y, Y.shape == (25,), Y.dtype == 'int32'
             where each element is 11-th element to the corresponding 10-symbol sequence from X
    
    
    PLEASE MAKE SURE that y symbols are indeed going immediately after X sequences, 
        since it is hard to debug later (NN will train, but it will generate something useless)
        
    The simplest approach is to first sample a matrix [n_batches, seq_len+1] 
        and than split it into X (first seq_len columns) and y (last column)


    There will be some tests for this function, but they won't cover everything
    """
    X_batch = []
    y_batch = []
    for first in range(n_batches):
        start = np.random.randint(len(source) - seq_len)
        x = source[start: start + seq_len]
        y = source[start + 1: start + seq_len + 1]
        X_batch.append(x)
        y_batch.append(y)
    X_batch = np.array(X_batch, dtype='int32')
    y_batch = np.array(y_batch, dtype='int32')
    return X_batch, y_batch
    
    

In [50]:
sample_random_batches(list(range(10)), 3, 3)

(array([[2, 3, 4],
        [6, 7, 8],
        [2, 3, 4]], dtype=int32), array([[3, 4, 5],
        [7, 8, 9],
        [3, 4, 5]], dtype=int32))

# Constants

In [14]:
#Training sequence length (truncation depth in BPTT)
seq_length = 5

#max gradient between recurrent layer applications (do not forget to use it)
grad_clip = 5




# Input variables

In [67]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.matrix('target y', 'int32')


# Build the neural network

You will need to define a neural network that processes a sequence of n tokens and outputs probabilities for n+1'st one.

The default architecture pattern would be


* Input
* Input processing (embedding/1-hot)
* Recurrent layer(s)
* Slicing last state
* Regular (e.g. dense) layers from last state
* output layer that predicts probabilities of next token





One way of data processing is to use an EmbeddingLayer (see previous seminar)

Alternatively, one could use a One-hot encoder
```
#One-hot encoding sketch
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# Can be applied to input_sequence - and the l_in below will require a new shape
# can also be used via ExpressionLayer(l_in, to_one_hot, shape_after_one_hot) - keeping l_in as it is
```



To cut out the last RNN state, use one of those
* `lasagne.layers.SliceLayer(rnn, -1, 1)`
* only_return_final=True in RNN params

In [86]:
from lasagne.layers import InputLayer,DenseLayer,EmbeddingLayer,ConcatLayer
from lasagne.layers import RecurrentLayer,LSTMLayer,GRULayer,CustomRecurrentLayer

In [119]:

l_in = InputLayer(shape=(None, None),input_var=input_sequence)

emb = EmbeddingLayer(l_in,
                     input_size=len(token_to_id),
                     output_size=64)
lstm1 = LSTMLayer(emb, 30, grad_clipping=grad_clip)
conc1 = ConcatLayer([lstm1, emb], axis=2)
lstm2 = LSTMLayer(conc, 60, grad_clipping=grad_clip)
conc2 = ConcatLayer([lstm2, emb], axis=2)

l_out = RecurrentLayer(conc2, len(token_to_id), 
                       nonlinearity=lasagne.nonlinearities.softmax,
                       grad_clipping=grad_clip,)




In [120]:
# Model weights
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print(weights)

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W]


In [121]:
network_output = lasagne.layers.get_output(l_out)
#If you use dropout do not forget to create deterministic version for evaluation

In [122]:
wow_loss = (lasagne.objectives.categorical_crossentropy(
        network_output.reshape((-1, len(token_to_id))), target_values.flatten()))
loss = wow_loss[4:].mean()

updates = lasagne.updates.adam(loss,weights)


# Compiling it

In [123]:

#training
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#computing loss without training
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# next character probabilities
probs = theano.function([input_sequence],network_output[:, -1, :],allow_input_downcast=True)


# Law generation

* We shall repeatedly apply NN to it's output.
 * Start with some sequence of length <seq length>
 * call probs(that sequence)
 * choose next symbol based on probs
 * append it to the sequence
 * remove the 0-th symbol so that it's length equals <seq length> again

* There are several policies of character picking
 * random, proportional to the probabilities
 * only take the one with highest probability
 * random, proportional to softmax(probas*alpha), where alpha is "greed" (from 0 to infinity)

In [124]:
def max_sample_fun(probs):
    """i generate the most likely symbol"""
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """i generate the next int32 character id randomly, proportional to probabilities
    
    probs - array of probabilities for every token
    
    you have to output a single integer - next token id - based on probs
    """
    threshold = np.random.uniform()
    current = 0
    for index, prob in enumerate(probs):
        current += prob
        if current >= threshold:
            return index






In [142]:
HYB = 0.8
def hybrid_sample_fun(probs):
    if np.random.uniform() > HYB:
        return proportional_sample_fun(probs)
    else:
        return max_sample_fun(probs)

In [125]:

def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
        
    parameters:
        sample_fun - max_ or proportional_sample_fun or whatever else you implemented
        
        The phrase is set using the variable seed_phrase

        The optional input "N" is used to set the number of characters of text to predict.     
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print("Using random seed:",seed_phrase)
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is str
        
        
    sample_ix = []
    x = list(map(lambda c: token_to_id.get(c,0), seed_phrase))
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)


# Model training

Here you can tweak parameters or insert your generation function


__Once something word-like starts generating, try increasing seq_length__



In [126]:
def training(n_epochs=100, batches_per_epoch=1000, batch_size=100):
    print("Training ...")


    #total N iterations
    #n_epochs=100

    # how many minibatches are there in the epoch 
    #batches_per_epoch = 1000

    #how many training sequences are processed in a single function call
    #batch_size=100


    for epoch in range(n_epochs):

        print ("Text generated proportionally to probabilities")
        generate_sample(proportional_sample_fun,None)

        print ("Text generated by picking most likely letters")
        generate_sample(max_sample_fun,None)

        avg_cost = 0;

        for _ in range(batches_per_epoch):

            x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
            avg_cost += train(x, y)

        print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))



In [127]:
seq_length = 8
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: пересече
----
 пересече2,У„бТцNэ№ФЮ©
УЖчХиСГ)т 8„7НГц+ХVа5cп0А«0@КЛ7ААшу©Ч7еДЛ04гьь–рЭь
–оп3§VмТ–НМдтэЙОНБМды+Ж–Э2аcлЙ.жйи9чяP 3%Ю"юЛ1;!ч5 шКаОУ+иЦ
П"беОАо.ьъеКнйю«%2PФ©:"Ыг"ПюбР6фихУь Сж!ДЭИ/х6„,ъЮ(5+М4(ы.А85–:Вж оТАд6ЦмN; 
----
Text generated by picking most likely letters
Using random seed: ля рассм
----
 ля рассм22222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222 
----
Epoch 0 average loss = 3.4602670741081236
Text generated proportionally to probabilities
Using random seed: Срок иск
----
 Срок иск интеашелзад енкенин
 кнерыиФаглнаовнияецамяви1онсдоае вомшозесжланррнилем нри ор Сорераединапееоли тнаеппистораторизеютареме (нимилнаич саад нсачоимсжиаавамле се угцеенвнеыратсгякрлсеикмлораудооь о л 
----
Text generated by picking most likely letters
Using random

In [128]:
seq_length = 16
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: подлежащего пере
----
 подлежащего передипличергит.
 Провефесли искоными, товарами усучей исполности 15.22.202 Предосламожных полное иле у рефистрочнновесье, и не о сть дпрешений доляважна.
 Статьнных кащитствельных резнечи для собъерумии  
----
Text generated by picking most likely letters
Using random seed: ртных средств пр
----
 ртных средств при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при пр 
----
Epoch 0 average loss = 1.6619688844680787
Text generated proportionally to probabilities
Using random seed: иально-техническ
----
 иально-техническолизакжнея

 1. Светства, и основание мущили


 199.
 Глию дугих с торуржа от вруглиц, возмията, устаков; на доспорушениях, располыженного товародации беспоривенки драможеннымися завлостьи Росса.

 1. 
----
Text gener

In [129]:
seq_length = 32
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: , о чем орган связи проинформиро
----
 , о чем орган связи проинформиро, иносте, товаровленамоборомо обязанной судомх порядной мильзающие полученного оргаках, размеров оспонтов комерющих в ред. Федерации удовленных очки:
 Суда;
 1) настово-к договоров грратнесних он стат 
----
Text generated by picking most likely letters
Using random seed: тели;
 2) государственные жилищн
----
 тели;
 2) государственные жилищным правонарушения и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и при принятия и  
----
Epoch 0 average loss = 1.358278728723526
Text generated proportionally to probabilities
Using random seed: правила о соединении и разъедине
----
 правила о соединении и разъединения юридических лиц – от иножения

 Статья 27.
 Авщитекса.
 В сили вещеенников основать обязанный подтном перящего, компуж

In [130]:
seq_length = 64
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: овершения или предмета административного правонарушения;
 4) кон
----
 овершения или предмета административного правонарушения;
 4) конкальные саходиям рассматривать) жизов

 Развершиях по условия при прешун правила долнений пунктом 2484) платить праве орган.
 2. Арбитражных в законом.

 Статья 19.
 Срорыми издубленном составление на 
----
Text generated by picking most likely letters
Using random seed: лица, управомоченного на получение вещи, вознаграждение за наход
----
 лица, управомоченного на получение вещи, вознаграждение за находящих предусмотренных правонарушения предусмотренных правонарушения предусмотренных правонарушения предусмотренных правонарушения предусмотренных правонарушения предусмотренных правонарушения предусмот 
----
Epoch 0 average loss = 1.2073253786563873
Text generated proportionally to probabilities
Using random seed: ья 56.
 Решение таможенного органа по жалобе на решение, действи

In [131]:
seq_length = 128
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: ия по решению суда на общее имущество может быть установлена долевая собственность этих лиц.

 Статья 245.
 Определение долей в 
----
 ия по решению суда на общее имущество может быть установлена долевая собственность этих лиц.

 Статья 245.
 Определение долей в ими.
 Если лицам, если оторативного шестьи 3 статьи 5) начал судебно о в зтоварительный места в размере, отказанных вещи, врехнию любой эти периванции, прителисяри общее бредительных вправе с матели.  
----
Text generated by picking most likely letters
Using random seed: 
 Статья 89

 Президент Российской Федерации:
 а) решает вопросы гражданства Российской Федерации и предоставления политического
----
 
 Статья 89

 Президент Российской Федерации:
 а) решает вопросы гражданства Российской Федерации и предоставления политического суда об административного штрафа на должностных лиц – от двухсот до двадцати до двухсот минимальных размеров оплаты труда;

In [138]:
seq_length = 256
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: яющим страну происхождения товаров, в соответствии с настоящим Кодексом может служить декларация о происхождении товара, составленная в произвольной форме, при условии, что в ней указаны сведения, позволяющие определить страну происхождения товаров. В каче
----
 яющим страну происхождения товаров, в соответствии с настоящим Кодексом может служить декларация о происхождении товара, составленная в произвольной форме, при условии, что в ней указаны сведения, позволяющие определить страну происхождения товаров. В качества обязательну, должностного закона пунцация, направляет взает неже соответствующего отсростатьи, и имей федеральным зрактом Российской Федерации, конфискать заверительных назначенам уплате в часть  
----
Text generated by picking most likely letters
Using random seed: ование электрической, тепловой энергии, нефти, газа или нефтепродуктов –
 влечет наложение административного штрафа на граждан в р

In [141]:
seq_length = 256
training(n_epochs=10, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: товить копию программы для ЭВМ или базы данных при условии, что эта копия предназначена только для архивных целей и для замены правомерно приобретенного экземпляра в случаях, когда оригинал программы для ЭВМ или базы данных утерян, уничтожен или стал непри
----
 товить копию программы для ЭВМ или базы данных при условии, что эта копия предназначена только для архивных целей и для замены правомерно приобретенного экземпляра в случаях, когда оригинал программы для ЭВМ или базы данных утерян, уничтожен или стал неприилось на срок обязан защиты хранитель или его территорий о встукции, рассматривается специали началосовленных времени жалобе

 Нарушение оборудования в части 7 настоящего Кодекса.
 3. В случыть вность 
----
Text generated by picking most likely letters
Using random seed: ебования ломбарда к залогодателю (должнику) погашаются, даже если сумма, вырученная при реализации заложенного имущества, недостат

np.savez('model.npz', *lasagne.layers.get_all_param_values(l_out))

In [151]:
seq_length = 256
training(n_epochs=100, batches_per_epoch=100, batch_size=1000)

Training ...
Text generated proportionally to probabilities
Using random seed: енных объединений материалы, содержащие данные, указывающие на наличие события административного правонарушения;
 3) сообщения и заявления физических и юридических лиц, а также сообщения в средствах массовой информации, содержащие данные, указывающие на на
----
 енных объединений материалы, содержащие данные, указывающие на наличие события административного правонарушения;
 3) сообщения и заявления физических и юридических лиц, а также сообщения в средствах массовой информации, содержащие данные, указывающие на наследником к нарушении уведовлетвоведения стоимости законом, к раздел.
 2. В случае устанавливается


 Статья 286.
 Предоставления предварительному.
 2. Переработки толки, установлено договором арансто 
----
Text generated by picking most likely letters
Using random seed: тентные органы или организации в Российской Федерации и за границей либо привлечь экспертов.
 Лица, участвующие в деле, могут пред

np.savez('model100.npz', *lasagne.layers.get_all_param_values(l_out))

# A chance to speed up training and get bonus score
* Try predicting next token probas at ALL ticks (like in the seminar part)
* much more objectives, much better gradients
* You may want to zero-out loss for first several iterations

# The New World Order

In [180]:
HYB = 0.7
seq_length = 128

In [181]:
seed = u"Каждый человек должен" #if you are using non-russian text corpora, use seed in it's language instead
sampling_fun = hybrid_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
                                                                                                            Каждый человек должен и морских вред.

 Статья 1007.
 Отвод в порядке в публичном моровику.
 2. Присы

 1. Правила такой суммы в соответствии с Правительства от трудового представительства и в соответствующем делам и принято решение в соответствии с законодательством Российской Федерации и оспущер таможенного органа или 
----


In [182]:
seed = u"В случае неповиновения"
sampling_fun = hybrid_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
                                                                                                           В случае неповиновения в Российской Федерации не момента в порядку возникающих в орезнавленное в полодовлечения или расчронных средств и представление по делу об административном правонарушении, принятия, предусмотренные статьей 214 настоящего Кодекса, подтверждающие обязанности производится в соответствии с законодатель 
----


In [ ]:
And so on at your will